# sweep

> Set of functions for querying an area of OLC grids

In [ ]:
# | default_exp sweep

In [ ]:
# | hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
# | export
import pandas as pd
from openlocationcode import openlocationcode as olc
from olc_utils.grid import *
from typing import Tuple

Given a string of characters representing X or Y characters of two OLC grids, find the characters of max and min grid

In [ ]:
# | export
def get_max_min_chars(
    str1: str,
    str2: str,
) -> Tuple[str, str]:
    max_str = ""
    min_str = ""

    for i in range(len(str1)):
        if str1[i] == str2[i]:
            max_str += str1[i]
            min_str += str1[i]

        elif olc.CODE_ALPHABET_.index(str1[i]) > olc.CODE_ALPHABET_.index(str2[i]):
            max_str += str1[i:]
            min_str += str2[i:]

            break
        else:
            max_str += str2[i:]
            min_str += str1[i:]

            break

    return (
        max_str,
        min_str,
    )

In [ ]:
test_eq(get_max_min_chars("856X", "8W6V"), ("8W6V", "856X"))
test_eq(get_max_min_chars("8W44", "8W3X"), ("8W44", "8W3X"))

Given two OLC grids, find the 4 corner grids in the rectangle defined by the two grids

In [ ]:
# | export
def get_four_corner_grids(
    grid1: str,
    grid2: str,
) -> Tuple[str, str]:
    if len(grid1) != len(grid2):
        raise ValueError("Grids must be of the same length")

    # Get the characters for X axis
    x_grid_1, y_grid_1 = olc_to_xy_chars(grid1)
    x_grid_2, y_grid_2 = olc_to_xy_chars(grid2)
    
    # Calculate the X max & min characters
    x_max_grid, x_min_grid = get_max_min_chars(x_grid_1, x_grid_2)
    y_max_grid, y_min_grid = get_max_min_chars(y_grid_1, y_grid_2)

    left_bottom = xy_chars_to_olc(x_min_grid, y_min_grid)
    left_top = xy_chars_to_olc(x_min_grid, y_max_grid)
    right_top = xy_chars_to_olc(x_max_grid, y_max_grid)
    right_bottom = xy_chars_to_olc(x_max_grid, y_min_grid)

    return left_bottom, left_top, right_top, right_bottom

In [ ]:
test_eq(
    get_four_corner_grids("9G2242WJ", "9G2242PW"), ('9G2242PJ', '9G2242WJ', '9G2242WW', '9G2242PW')
)
test_eq(
    get_four_corner_grids('9G2542WW', "9X2242PJ"), ('9G2542PW', '9G2542WW', '9X2242WJ', '9X2242PJ')
)

Given two OLC grids, get all OLC grids in the rectangle formed between them

In [ ]:
#| export
def get_all_olc_between_grids(
    grid1: str,
    grid2: str,
) -> pd.DataFrame:
    # Get the four corners of the grids
    left_bottom, _, right_top, _ = get_four_corner_grids(
        grid1, grid2
    )

    # Get X and Y offsets
    x_offset, y_offset = get_offsets_between_grids(left_bottom, right_top)

    # Get all the grids between the two grids
    all_olc = []

    for j in range(y_offset + 1):
        all_olc.extend(
            get_olc_with_offsets(left_bottom, i, j)
            for i in range(x_offset + 1)
        )
    return all_olc

In [ ]:
olc_list = get_all_olc_between_grids("9G2242WJ", "9G225322")
print(len(olc_list))
display(olc_list)

27


['9G2242WJ',
 '9G2242WM',
 '9G2242WP',
 '9G2242WQ',
 '9G2242WR',
 '9G2242WV',
 '9G2242WW',
 '9G2242WX',
 '9G2243W2',
 '9G2242XJ',
 '9G2242XM',
 '9G2242XP',
 '9G2242XQ',
 '9G2242XR',
 '9G2242XV',
 '9G2242XW',
 '9G2242XX',
 '9G2243X2',
 '9G22522J',
 '9G22522M',
 '9G22522P',
 '9G22522Q',
 '9G22522R',
 '9G22522V',
 '9G22522W',
 '9G22522X',
 '9G225322']

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()